<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/EqBalancing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import re
import numpy as np
import pandas as pd

In [73]:
rxn ='aKNO3 + C12H22O11  + 2C6H14O6 -> bCO2 + cH2O + dN2 + eK2CO3 '
rxn ='aKNO3 + 5C12H22O11 -> bCO2 + cH2O + dN2 + eK2CO3 '

In [74]:
stoic_molecule_pat=re.compile(r'(.*?)([A-Z].*)')
atom_num_pat = re.compile(r'([A-Z][a-z]?)(\d*)')
rxn=re.sub(r'\s+','',rxn)
lhs,rhs=rxn.split('->')

In [75]:
def process(side,sign):
    molecule_dict={}
    for stoic_molecule in side.split('+'):
        stoic,molecule=stoic_molecule_pat.match(stoic_molecule).group(1,2)
        stoic = sign*1 if stoic=='' else int(stoic)*sign if stoic.isnumeric() else stoic 
        d={'stoic':stoic}
        for atom,num in atom_num_pat.findall(molecule):
            num = 1 if num=='' else int(num)
            d[atom]=num
        molecule_dict[molecule]=d
    df=pd.DataFrame(molecule_dict)
    df=df.fillna(0)
    return df

In [76]:
df=pd.concat([process(lhs,-1),process(rhs,1)],axis=1)
df

,KNO3,C12H22O11,CO2,H2O,N2,K2CO3
stoic,a,-5.0,b,c,d,e
K,1,0.0,0,0,0,2
N,1,0.0,0,0,2,0
O,3,11.0,2,1,0,3
C,0,12.0,1,0,0,1
H,0,22.0,0,2,0,0


In [77]:
fixed_col=pd.to_numeric(df.loc['stoic',:],errors='coerce').notna()
atom_num = df.iloc[1:,:].astype(int)


In [78]:
rhs=-atom_num.loc[:,fixed_col] @ df.loc['stoic',fixed_col].astype(float)

In [79]:
res=np.linalg.solve(atom_num.loc[:,~fixed_col], rhs)

In [80]:
df.loc['stoic',~fixed_col]=res

In [81]:
df

,KNO3,C12H22O11,CO2,H2O,N2,K2CO3
stoic,-48.0,-5.0,36.0,55.0,24.0,24.0
K,1,0.0,0,0,0,2
N,1,0.0,0,0,2,0
O,3,11.0,2,1,0,3
C,0,12.0,1,0,0,1
H,0,22.0,0,2,0,0


In [82]:
lhs_rxn=[]
rhs_rxn=[]
for stoic,molecule in zip(df.loc['stoic',:], df.columns):
    stoic = int(stoic) if abs(int(stoic)-stoic)<1e-10 else stoic
    if stoic<0:
        if abs(stoic)==1:
            lhs_rxn.append(f'{molecule}')
        else:
            lhs_rxn.append(f'{abs(stoic)}{molecule}')
    else:
        if abs(stoic)==1:
            rhs_rxn.append(f'{molecule}')
        rhs_rxn.append(f'{stoic}{molecule}')
        
' + '.join(lhs_rxn) + ' -> ' + ' + '.join(rhs_rxn)

'48KNO3 + 5C12H22O11 -> 36CO2 + 55H2O + 24N2 + 24K2CO3'